<a href="https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/docs/examples/llm/fireworks_cookbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在 Colab 中打开"/></a>


# Fireworks函数调用手册

Fireworks.ai支持对其LLMs进行函数调用，类似于OpenAI。这使用户可以直接描述可用的工具/函数集，并使模型动态选择正确的函数调用来调用，而无需用户复杂的提示。

由于我们的Fireworks LLM直接是OpenAI的子类，因此我们可以在Fireworks中使用现有的抽象。

我们将在三个层面上展示这一点：直接在模型API上，作为Pydantic程序的一部分（结构化输出提取），以及作为代理的一部分。


In [ ]:
%pip install llama-index-llms-fireworks

In [ ]:
%pip install llama-index

In [ ]:
import os

os.environ["FIREWORKS_API_KEY"] = ""

In [ ]:
from llama_index.llms.fireworks import Fireworks

## 定义fireworks模型
llm = Fireworks(
    model="accounts/fireworks/models/firefunction-v1", temperature=0
)

/Users/jerryliu/Programming/gpt_index/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 在LLM模块上调用函数

您可以直接在LLM模块上输入函数调用。


In [ ]:
from pydantic import BaseModel
from llama_index.llms.openai.utils import to_openai_tool


class Song(BaseModel):
    """具有名称和艺术家的歌曲"""

    name: str
    artist: str


# 这将pydantic模型转换为函数，以提取结构化输出
song_fn = to_openai_tool(Song)


response = llm.complete("从Beyonce生成一首歌曲", tools=[song_fn])
tool_calls = response.additional_kwargs["tool_calls"]
print(tool_calls)

[ChatCompletionMessageToolCall(id='call_34ZaM0xPl1cveODjVUpO78ra', function=Function(arguments='{"name": "Crazy in Love", "artist": "Beyonce"}', name='Song'), type='function', index=0)]


## 使用Pydantic程序

我们的Pydantic程序允许将结构化输出提取到Pydantic对象中。`OpenAIPydanticProgram`利用函数调用来进行结构化输出提取。


In [ ]:
from llama_index.program.openai import OpenAIPydanticProgram

In [ ]:
prompt_template_str = "Generate a song about {artist_name}"
program = OpenAIPydanticProgram.from_defaults(
    output_cls=Song, prompt_template_str=prompt_template_str, llm=llm
)

In [ ]:
output = program(artist_name="Eminem")

In [ ]:
output

Song(name='Rap God', artist='Eminem')

在本示例中，我们将使用OpenAI的GPT-3代理来生成文本。我们将演示如何设置代理并使用它来生成文本。


In [ ]:
from llama_index.agent.openai import OpenAIAgent

In [ ]:

# from llama_index.core.tools import BaseTool,FunctionTool

def multiply(a: int, b: int) -> int:
    """将两个整数相乘，并返回结果整数"""
    return a * b

multiply_tool = FunctionTool.from_defaults(fn=multiply)

def add(a: int, b: int) -> int:
    """将两个整数相加，并返回结果整数"""
    return a + b

add_tool = FunctionTool.from_defaults(fn=add)

In [ ]:
agent = OpenAIAgent.from_tools(
    [multiply_tool, add_tool], llm=llm, verbose=True
)

In [ ]:
response = agent.chat("What is (121 * 3) + 42?")
print(str(response))

Added user message to memory: What is (121 * 3) + 42?
=== Calling Function ===
Calling function: multiply with args: {"a": 121, "b": 3}
Got output: 363

=== Calling Function ===
Calling function: add with args: {"a": 363, "b": 42}
Got output: 405

The result of (121 * 3) + 42 is 405.
